## https://towardsdatascience.com/a-brief-introduction-to-intent-classification-96fda6b1f557

In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
import re
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


In [9]:
def load_dataset(filename):
  df = pd.read_csv(filename, encoding = "utf8", sep='\t',names = ["Sentence", "Intent"])
  print(df.head())
  intent = df["Intent"]
  unique_intent = list(set(intent))
  sentences = list(df["Sentence"])
  
  return (intent, unique_intent, sentences)
  


In [10]:
intent, unique_intent, sentences = load_dataset("intent_data.csv")

                                            Sentence             Intent
0               - Bakabileceğim bazı dersler var mı?  ask_faq_tutorials
1                               - dersleriniz var mı  ask_faq_tutorials
2  - ciftci_bot'un nasıl kullanılacağı hakkında b...  ask_faq_tutorials
3                               - ciftci_bot eğitimi  ask_faq_tutorials
4                 - bana öğretici gösterebilir misin  ask_faq_tutorials


In [11]:
print(sentences[:5])

['- Bakabileceğim bazı dersler var mı?', '- dersleriniz var mı', "- ciftci_bot'un nasıl kullanılacağı hakkında bir eğiticiye ihtiyacım var", '- ciftci_bot eğitimi', '- bana öğretici gösterebilir misin']


In [12]:
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/zerocode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/zerocode/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
#define stemmer
stemmer = LancasterStemmer()

In [14]:
def cleaning(sentences):
  words = []
  for s in sentences:
    clean = re.sub(r'[^ a-z A-Z 0-9]', " ", s)
    w = word_tokenize(clean)
    #stemming
    words.append([i.lower() for i in w])
    
  return words  

In [15]:
cleaned_words = cleaning(sentences)
print(len(cleaned_words))
print(cleaned_words[:2])  
  


994
[['bakabilece', 'im', 'baz', 'dersler', 'var', 'm'], ['dersleriniz', 'var', 'm']]


In [16]:
def create_tokenizer(words, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'):
  token = Tokenizer(filters = filters)
  token.fit_on_texts(words)
  return token

In [17]:
def max_length(words):
  return(len(max(words, key = len)))
  

In [18]:
word_tokenizer = create_tokenizer(cleaned_words)
vocab_size = len(word_tokenizer.word_index) + 1
max_length = max_length(cleaned_words)

print("Vocab Size = %d and Maximum length = %d" % (vocab_size, max_length))

Vocab Size = 1106 and Maximum length = 21


In [19]:
def encoding_doc(token, words):
  return(token.texts_to_sequences(words))

In [20]:
encoded_doc = encoding_doc(word_tokenizer, cleaned_words)

In [21]:
def padding_doc(encoded_doc, max_length):
  return(pad_sequences(encoded_doc, maxlen = max_length, padding = "post"))

In [22]:
padded_doc = padding_doc(encoded_doc, max_length)

In [0]:
padded_doc[:5]

array([[ 25,  77, 332,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [ 25,  77,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  1,  25, 198, 181,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [ 51,  10,  77,  16,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  8, 268,   4,  10,  30,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int32)

In [23]:
print("Shape of padded docs = ",padded_doc.shape)

Shape of padded docs =  (994, 21)


In [24]:
#tokenizer with filter changed
output_tokenizer = create_tokenizer(unique_intent, filters = '!"#$%&()*+,-/:;<=>?@[\]^`{|}~')


In [25]:
output_tokenizer.word_index

{'ask_whoisit': 1,
 'signup_newsletter': 2,
 'location': 3,
 'ask_time': 4,
 'ask_whatismyname': 5,
 'contact_sales': 6,
 'enter_data': 7,
 'ask_whatspossible': 8,
 'ask_howdoing': 9,
 'ask_whoami': 10,
 'ask_builder': 11,
 'ask_wherefrom': 12,
 'thank': 13,
 'ask_howold': 14,
 'bye': 15,
 'affirm': 16,
 'ask_weather': 17,
 'ask_faq_tutorials': 18,
 'deny': 19,
 'canthelp': 20,
 'out_of_scope': 21,
 'greet': 22,
 'ask_isbot': 23,
 'human_handoff': 24}

In [26]:
encoded_output = encoding_doc(output_tokenizer, intent)

In [27]:
encoded_output = np.array(encoded_output).reshape(len(encoded_output), 1)

In [28]:
encoded_output.shape

(994, 1)

In [29]:
def one_hot(encode):
  o = OneHotEncoder(sparse = False)
  return(o.fit_transform(encode))

In [30]:
output_one_hot = one_hot(encoded_output)

/home/zerocode/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [31]:
output_one_hot.shape

(994, 24)

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
train_X, val_X, train_Y, val_Y = train_test_split(padded_doc, output_one_hot, shuffle = True, test_size = 0.2)


In [34]:
print("Shape of train_X = %s and train_Y = %s" % (train_X.shape, train_Y.shape))
print("Shape of val_X = %s and val_Y = %s" % (val_X.shape, val_Y.shape))

Shape of train_X = (795, 21) and train_Y = (795, 24)
Shape of val_X = (199, 21) and val_Y = (199, 24)


In [38]:
def create_model(vocab_size, max_length):
  model = Sequential()
  model.add(Embedding(vocab_size, 128, input_length = max_length, trainable = False))
  model.add(Bidirectional(LSTM(128)))
#   model.add(LSTM(128))
  model.add(Dense(32, activation = "relu"))
  model.add(Dropout(0.5))
  model.add(Dense(24, activation = "softmax"))
  
  return model

In [39]:
model = create_model(vocab_size, max_length)

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 21, 128)           141568    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               263168    
_________________________________________________________________
dense_3 (Dense)              (None, 32)                8224      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 24)                792       
Total params: 413,752
Trainable params: 272,184
Non-trainable params: 141,568
_________________________________________________________________


In [41]:
filename = 'model_omid.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

hist = model.fit(train_X, train_Y, epochs = 500, batch_size = 32, validation_data = (val_X, val_Y), callbacks = [checkpoint])

Train on 795 samples, validate on 199 samples
Epoch 1/500
795/795 [==============================] - 0s 622us/step - loss: 0.7988 - acc: 0.7597 - val_loss: 2.4459 - val_acc: 0.4874

Epoch 00001: val_loss improved from inf to 2.44589, saving model to model_omid.h5
Epoch 2/500
795/795 [==============================] - 0s 626us/step - loss: 0.7884 - acc: 0.7660 - val_loss: 2.7007 - val_acc: 0.5226

Epoch 00002: val_loss did not improve from 2.44589
Epoch 3/500
795/795 [==============================] - 1s 638us/step - loss: 0.7203 - acc: 0.7786 - val_loss: 2.8644 - val_acc: 0.4925

Epoch 00003: val_loss did not improve from 2.44589
Epoch 4/500
795/795 [==============================] - 1s 635us/step - loss: 0.6522 - acc: 0.7836 - val_loss: 2.7571 - val_acc: 0.5075

Epoch 00004: val_loss did not improve from 2.44589
Epoch 5/500
795/795 [==============================] - 1s 646us/step - loss: 0.6016 - acc: 0.8050 - val_loss: 2.6693 - val_acc: 0.4874

Epoch 00005: val_loss did not improve f

795/795 [==============================] - 1s 747us/step - loss: 0.4278 - acc: 0.8629 - val_loss: 3.2952 - val_acc: 0.5327

Epoch 00044: val_loss did not improve from 2.44589
Epoch 45/500
795/795 [==============================] - 1s 1ms/step - loss: 0.4427 - acc: 0.8553 - val_loss: 3.2904 - val_acc: 0.5176

Epoch 00045: val_loss did not improve from 2.44589
Epoch 46/500
795/795 [==============================] - 1s 865us/step - loss: 0.4184 - acc: 0.8604 - val_loss: 3.3047 - val_acc: 0.5226

Epoch 00046: val_loss did not improve from 2.44589
Epoch 47/500
795/795 [==============================] - 1s 793us/step - loss: 0.4002 - acc: 0.8755 - val_loss: 3.3001 - val_acc: 0.5276

Epoch 00047: val_loss did not improve from 2.44589
Epoch 48/500
795/795 [==============================] - 1s 644us/step - loss: 0.4212 - acc: 0.8642 - val_loss: 2.8177 - val_acc: 0.5025

Epoch 00048: val_loss did not improve from 2.44589
Epoch 49/500
795/795 [==============================] - 1s 645us/step - los


Epoch 00087: val_loss did not improve from 2.44589
Epoch 88/500
795/795 [==============================] - 1s 669us/step - loss: 0.3312 - acc: 0.8943 - val_loss: 3.6666 - val_acc: 0.5226

Epoch 00088: val_loss did not improve from 2.44589
Epoch 89/500
795/795 [==============================] - 1s 665us/step - loss: 0.3259 - acc: 0.8994 - val_loss: 3.7050 - val_acc: 0.4975

Epoch 00089: val_loss did not improve from 2.44589
Epoch 90/500
795/795 [==============================] - 1s 860us/step - loss: 0.3056 - acc: 0.8969 - val_loss: 3.6724 - val_acc: 0.5176

Epoch 00090: val_loss did not improve from 2.44589
Epoch 91/500
795/795 [==============================] - 1s 923us/step - loss: 0.3041 - acc: 0.8994 - val_loss: 3.7996 - val_acc: 0.5025

Epoch 00091: val_loss did not improve from 2.44589
Epoch 92/500
795/795 [==============================] - 1s 721us/step - loss: 0.3072 - acc: 0.8994 - val_loss: 4.1318 - val_acc: 0.4874

Epoch 00092: val_loss did not improve from 2.44589
Epoch 93

795/795 [==============================] - 1s 677us/step - loss: 0.2391 - acc: 0.9220 - val_loss: 3.8130 - val_acc: 0.5276

Epoch 00131: val_loss did not improve from 2.44589
Epoch 132/500
795/795 [==============================] - 1s 657us/step - loss: 0.2294 - acc: 0.9208 - val_loss: 3.9912 - val_acc: 0.5427

Epoch 00132: val_loss did not improve from 2.44589
Epoch 133/500
795/795 [==============================] - 1s 740us/step - loss: 0.2536 - acc: 0.9069 - val_loss: 4.0054 - val_acc: 0.5327

Epoch 00133: val_loss did not improve from 2.44589
Epoch 134/500
795/795 [==============================] - 1s 679us/step - loss: 0.2297 - acc: 0.9245 - val_loss: 4.1676 - val_acc: 0.5176

Epoch 00134: val_loss did not improve from 2.44589
Epoch 135/500
795/795 [==============================] - 1s 673us/step - loss: 0.3497 - acc: 0.9157 - val_loss: 4.2653 - val_acc: 0.5025

Epoch 00135: val_loss did not improve from 2.44589
Epoch 136/500
795/795 [==============================] - 1s 723us/ste

795/795 [==============================] - 1s 675us/step - loss: 0.2142 - acc: 0.9182 - val_loss: 4.3345 - val_acc: 0.5176

Epoch 00174: val_loss did not improve from 2.44589
Epoch 175/500
795/795 [==============================] - 1s 657us/step - loss: 0.1715 - acc: 0.9396 - val_loss: 4.4008 - val_acc: 0.5226

Epoch 00175: val_loss did not improve from 2.44589
Epoch 176/500
795/795 [==============================] - 1s 657us/step - loss: 0.1711 - acc: 0.9459 - val_loss: 4.4382 - val_acc: 0.5276

Epoch 00176: val_loss did not improve from 2.44589
Epoch 177/500
795/795 [==============================] - 1s 660us/step - loss: 0.1628 - acc: 0.9497 - val_loss: 4.3700 - val_acc: 0.5427

Epoch 00177: val_loss did not improve from 2.44589
Epoch 178/500
795/795 [==============================] - 1s 651us/step - loss: 0.1595 - acc: 0.9459 - val_loss: 4.6486 - val_acc: 0.5226

Epoch 00178: val_loss did not improve from 2.44589
Epoch 179/500
795/795 [==============================] - 1s 660us/ste

...

795/795 [==============================] - 1s 670us/step - loss: 0.1015 - acc: 0.9673 - val_loss: 5.0456 - val_acc: 0.5276

Epoch 00303: val_loss did not improve from 2.44589
Epoch 304/500
795/795 [==============================] - 1s 683us/step - loss: 0.1154 - acc: 0.9610 - val_loss: 4.8251 - val_acc: 0.5025

Epoch 00304: val_loss did not improve from 2.44589
Epoch 305/500
795/795 [==============================] - 1s 784us/step - loss: 0.1288 - acc: 0.9522 - val_loss: 4.9666 - val_acc: 0.5126

Epoch 00305: val_loss did not improve from 2.44589
Epoch 306/500
795/795 [==============================] - 1s 662us/step - loss: 0.1264 - acc: 0.9497 - val_loss: 4.8787 - val_acc: 0.5126

Epoch 00306: val_loss did not improve from 2.44589
Epoch 307/500
795/795 [==============================] - 1s 787us/step - loss: 0.1038 - acc: 0.9623 - val_loss: 4.9130 - val_acc: 0.5176

Epoch 00307: val_loss did not improve from 2.44589
Epoch 308/500
795/795 [==============================] - 1s 691us/ste

795/795 [==============================] - 1s 701us/step - loss: 0.2137 - acc: 0.9572 - val_loss: 4.8754 - val_acc: 0.5176

Epoch 00346: val_loss did not improve from 2.44589
Epoch 347/500
795/795 [==============================] - 1s 676us/step - loss: 0.1686 - acc: 0.9509 - val_loss: 4.7075 - val_acc: 0.5226

Epoch 00347: val_loss did not improve from 2.44589
Epoch 348/500
795/795 [==============================] - 1s 693us/step - loss: 0.1266 - acc: 0.9560 - val_loss: 4.6279 - val_acc: 0.5176

Epoch 00348: val_loss did not improve from 2.44589
Epoch 349/500
795/795 [==============================] - 1s 696us/step - loss: 0.1168 - acc: 0.9547 - val_loss: 4.8130 - val_acc: 0.5276

Epoch 00349: val_loss did not improve from 2.44589
Epoch 350/500
795/795 [==============================] - 1s 686us/step - loss: 0.1014 - acc: 0.9711 - val_loss: 4.7353 - val_acc: 0.5276

Epoch 00350: val_loss did not improve from 2.44589
Epoch 351/500
795/795 [==============================] - 1s 688us/ste

795/795 [==============================] - 1s 679us/step - loss: 0.0838 - acc: 0.9736 - val_loss: 5.1770 - val_acc: 0.5176

Epoch 00389: val_loss did not improve from 2.44589
Epoch 390/500
795/795 [==============================] - 1s 661us/step - loss: 0.1017 - acc: 0.9610 - val_loss: 5.3162 - val_acc: 0.5176

Epoch 00390: val_loss did not improve from 2.44589
Epoch 391/500
795/795 [==============================] - 1s 656us/step - loss: 0.0928 - acc: 0.9648 - val_loss: 5.2369 - val_acc: 0.5176

Epoch 00391: val_loss did not improve from 2.44589
Epoch 392/500
795/795 [==============================] - 1s 657us/step - loss: 0.1077 - acc: 0.9560 - val_loss: 5.1776 - val_acc: 0.5176

Epoch 00392: val_loss did not improve from 2.44589
Epoch 393/500
795/795 [==============================] - 1s 655us/step - loss: 0.1040 - acc: 0.9623 - val_loss: 5.1956 - val_acc: 0.5276

Epoch 00393: val_loss did not improve from 2.44589
Epoch 394/500
795/795 [==============================] - 1s 657us/ste

795/795 [==============================] - 1s 685us/step - loss: 0.0782 - acc: 0.9711 - val_loss: 5.2072 - val_acc: 0.5025

Epoch 00432: val_loss did not improve from 2.44589
Epoch 433/500
795/795 [==============================] - 1s 823us/step - loss: 0.0974 - acc: 0.9585 - val_loss: 5.2296 - val_acc: 0.5126

Epoch 00433: val_loss did not improve from 2.44589
Epoch 434/500
795/795 [==============================] - 1s 656us/step - loss: 0.1104 - acc: 0.9547 - val_loss: 5.3109 - val_acc: 0.5126

Epoch 00434: val_loss did not improve from 2.44589
Epoch 435/500
795/795 [==============================] - 1s 657us/step - loss: 0.0672 - acc: 0.9761 - val_loss: 5.2660 - val_acc: 0.5226

Epoch 00435: val_loss did not improve from 2.44589
Epoch 436/500
795/795 [==============================] - 1s 655us/step - loss: 0.0910 - acc: 0.9698 - val_loss: 5.3617 - val_acc: 0.5176

Epoch 00436: val_loss did not improve from 2.44589
Epoch 437/500
795/795 [==============================] - 1s 655us/ste

795/795 [==============================] - 1s 707us/step - loss: 0.1969 - acc: 0.9447 - val_loss: 4.8338 - val_acc: 0.5226

Epoch 00475: val_loss did not improve from 2.44589
Epoch 476/500
795/795 [==============================] - 1s 730us/step - loss: 0.1320 - acc: 0.9572 - val_loss: 4.7341 - val_acc: 0.5025

Epoch 00476: val_loss did not improve from 2.44589
Epoch 477/500
795/795 [==============================] - 1s 753us/step - loss: 0.1233 - acc: 0.9610 - val_loss: 5.2126 - val_acc: 0.4975

Epoch 00477: val_loss did not improve from 2.44589
Epoch 478/500
795/795 [==============================] - 1s 743us/step - loss: 0.1161 - acc: 0.9635 - val_loss: 4.9724 - val_acc: 0.5025

Epoch 00478: val_loss did not improve from 2.44589
Epoch 479/500
795/795 [==============================] - 1s 749us/step - loss: 0.1167 - acc: 0.9585 - val_loss: 4.5602 - val_acc: 0.5226

Epoch 00479: val_loss did not improve from 2.44589
Epoch 480/500
795/795 [==============================] - 1s 744us/ste

In [42]:
 model = load_model("model_omid.h5")

In [43]:
def predictions(text):
  clean = re.sub(r'[^ a-z A-Z 0-9]', " ", text)
  test_word = word_tokenize(clean)
  test_word = [w.lower() for w in test_word]
  test_ls = word_tokenizer.texts_to_sequences(test_word)
  print(test_word)
  #Check for unknown words
  if [] in test_ls:
    test_ls = list(filter(None, test_ls))
    
  test_ls = np.array(test_ls).reshape(1, len(test_ls))
 
  x = padding_doc(test_ls, max_length)
  
  pred = model.predict_proba(x)
  
  
  return pred


  

In [44]:
def get_final_output(pred, classes):
  predictions = pred[0]
 
  classes = np.array(classes)
  ids = np.argsort(-predictions)
  classes = classes[ids]
  predictions = -np.sort(-predictions)
 
  for i in range(pred.shape[1]):
    print("%s has confidence = %s" % (classes[i], (predictions[i])))



In [45]:
text = "nasılsın?"
pred = predictions(text)
get_final_output(pred, unique_intent)

['nas', 'ls', 'n']
ask_howdoing has confidence = 0.50010353
out_of_scope has confidence = 0.44888446
enter_data has confidence = 0.012501485
ask_whatspossible has confidence = 0.0068860273
ask_wherefrom has confidence = 0.006296458
ask_weather has confidence = 0.0059986585
ask_howold has confidence = 0.0030379593
ask_whoisit has confidence = 0.0030156288
ask_faq_tutorials has confidence = 0.0029205296
ask_whatismyname has confidence = 0.0022733742
thank has confidence = 0.0021527633
deny has confidence = 0.0016812265
ask_isbot has confidence = 0.0008999464
ask_builder has confidence = 0.0007925013
canthelp has confidence = 0.00076479354
bye has confidence = 0.00040852674
signup_newsletter has confidence = 0.00039647726
location has confidence = 0.00037827398
greet has confidence = 0.00027790072
affirm has confidence = 0.00018806348
contact_sales has confidence = 0.000116538875
ask_whoami has confidence = 1.2769889e-05
human_handoff has confidence = 1.102856e-05
ask_time has confidence 

In [50]:
text = "Bana Yardım Et "
pred = predictions(text)
get_final_output(pred, unique_intent)

['bana', 'yard', 'm', 'et']
ask_whatspossible has confidence = 0.47114953
out_of_scope has confidence = 0.19229442
ask_whatismyname has confidence = 0.17394577
ask_whoami has confidence = 0.044428747
ask_faq_tutorials has confidence = 0.036433663
canthelp has confidence = 0.033444274
human_handoff has confidence = 0.0120426975
ask_builder has confidence = 0.0073872474
contact_sales has confidence = 0.0072379983
ask_whoisit has confidence = 0.006612401
ask_weather has confidence = 0.005776042
ask_time has confidence = 0.001946482
affirm has confidence = 0.0015712946
enter_data has confidence = 0.0015385131
ask_howdoing has confidence = 0.0013053014
bye has confidence = 0.0008329927
ask_howold has confidence = 0.000589796
ask_wherefrom has confidence = 0.00047722156
deny has confidence = 0.00030871446
thank has confidence = 0.00026194536
ask_isbot has confidence = 0.0001981732
signup_newsletter has confidence = 0.00012686859
greet has confidence = 8.9193934e-05
location has confidence = 

In [52]:
text = "ankarada subesi var"
pred = predictions(text)
get_final_output(pred, unique_intent)

['ankarada', 'subesi', 'var']
enter_data has confidence = 0.4448211
out_of_scope has confidence = 0.22015561
location has confidence = 0.16338307
greet has confidence = 0.03231369
ask_wherefrom has confidence = 0.029007068
affirm has confidence = 0.019534191
ask_whoisit has confidence = 0.017577296
signup_newsletter has confidence = 0.015661875
ask_howdoing has confidence = 0.013410095
ask_isbot has confidence = 0.011735121
contact_sales has confidence = 0.010368941
deny has confidence = 0.0059149424
ask_faq_tutorials has confidence = 0.0042496724
ask_whatspossible has confidence = 0.004168474
ask_builder has confidence = 0.0025246206
bye has confidence = 0.0017891763
ask_time has confidence = 0.0012344839
thank has confidence = 0.0010668816
canthelp has confidence = 0.00026874102
ask_howold has confidence = 0.00022374465
human_handoff has confidence = 0.00020792043
ask_whatismyname has confidence = 0.00020105134
ask_whoami has confidence = 0.0001255375
ask_weather has confidence = 5.6